<h2>Building data pipelines for TensorFlow</h2>
<p>Building the input pipeline in a program or in any machine learning project is a lengthy process, sometimes it take more time than building the actual model. To build efficient pipeline for data (Images or Text) Tensorflow provides Dataset module called <b>tf.data</b>.<br>
tf.data API enables us to build complex input pipelines from simple, reusable pieces. It loads data (Images or Text) from the disk, applies transformations, then creates the batches and sends it to the GPU. <br>
To construct a dataset from data in memory, we use <b>tf.data.Dataset.from_tensors() </b> or <b>tf.data.Dataset.from_tensors_slices() </b>. If the input data stored in a file, TFRecord format we use <b>tf.data.TFRecordDataset()</b>.A simple example showing the construction of dataset. 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Importing tensorflow library
import tensorflow as tf

#Loading the list of 5 values to dataset
datast = tf.data.Dataset.from_tensor_slices([8,3,0,8,2])

#It will print the shape and type of the dataset
print(datast)

<TensorSliceDataset shapes: (), types: tf.int32>


In [7]:
#Importing necessary libraries
import collections
import pathlib
import re
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds


<h3>Load the text datasets</h3>
<p>To load the text files to the tensorflow we use <b>tf.data.TextLineDataset</b> and <b>tf.text</b> to preprocess the data for finer-gain control.<br>
We can load the txt file from the disk or we can download the text from online sources. 

In [8]:
#Here we are trying to load the simple txt file from drive
dataset = tf.data.TextLineDataset("//content/drive/MyDrive/Building  Data pipelines for Tensor Flow/tensorlow.txt")

#Print the content with shape and datatype
for line in dataset:
  print(line)

tf.Tensor(b'Hello, we are learning the tensorflow.', shape=(), dtype=string)
tf.Tensor(b'It is a essential concept to learn machine learning and deep learning.', shape=(), dtype=string)


<p>In this example, we will dowload a dataset of programming questions from StackOverflow. Every question in stackoverflow is labelled with one tag (it is Python, CSharp, Javascript etc).

In [9]:
#Downloading the text data from stackoverflow website
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')
dataset_dir = pathlib.Path(dataset).parent

In [10]:
#List the directories of the dataset
list(dataset_dir.iterdir())

train_dir = dataset_dir/'train'
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/README.md'),
 PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz'),
 PosixPath('/tmp/.keras/train')]

<p>The <b>train/python, train/java, train/csharp</b> etc. these directories contain number of text of files each having a question.</p>

In [11]:
#Loading the python dirictory named 1789.txt and displaying its contents
simple_file = train_dir/'python/1789.txt'
with open(simple_file) as f:
  print(f.read())

"how to install pydelicious? im using on my windows blank 3.3.2 i have a problem to instal pydelicious api. i was trying to make it work for a whole week but i couldnt make it happen..this is my problem:..c:usersbartldesktopblank-delicious-master&gt;blank setup.py install.traceback (most recent call last):.  file ""setup.py"", line 11, in &lt;module&gt;.    import pydelicious.  file ""c:usersbartldesktopblank-delicious-masterpydelicious__init__.py"", line 171.    raise deliciousitemexistserror, params['url'].                                  ^.syntaxerror: invalid syntax...can somebody help me?"



<p>We will load the data off disk and prepare it into the format suitable for training. For this task, we use <b>text_dataset_from_directory</b> utility to create a labeled tf.data.Dataset.<br>
The <b>preprocessing.text_dataset_from_directory</b>expects a directory structure as follows<br>
<i>train/<br>
...java/<br>
......1.txt<br>
......2.txt<br>
......3.txt<br>
...javascript/<br>
......1.txt<br>
......2.txt<br>
......3.txt<br></i></p>

<h3>Split the dataset into train, test and validation batches</h3>
<p>When running a machine learning program or experiment, it is good to divide the dataset into 3 sets <b>Train, Validation, Test</b>. The dataset we have loaded (Stackoverflow) has been divided into train and test but not validation. We create a validation set using an 80:20 split of the training data by using the arguement <b>validation_split</b> 

In [12]:
#Specifying the batch_size and seed values
batch_size = 32
seed = 42

#Preparing the dataset for training
train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training', #This can be changed to validation because of we dividng 80:20 ratio
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [13]:
#Preparing the dataset for validation by specifying the subset attribute to validation
val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


<p>For preparing the test dataset, validation_split and subset arguements are not required because validation set is already created by dividing the training dataset.  

In [14]:
#Preparing the dataset for testing
test_dir = dataset_dir/'test'
test_ds = preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


<h4>Prepare the dataset for training</h4>
<p>Before training the dataset we will standardize, tokenize and vectorize the data using <b>preprocessing.TextVectorization</b> layer.
<ul><li><b>Standardization</b> refers to preprocessing the text, typically to remove punctuation and HTML elements which will simplify the dataset.
Default standardization converts text to lowercase and removes punctuation.</li>
<li><b>Tokenization</b> refers to splitting the string in the datasets to tokens(for example, splitting a sentence into individual words by splitting on whitespace.
Default tokenizer split text on whitespaces.</li>
<li><b>Vectorization</b> refers to converting tokens to numbers, so they can be fed to neural network.
Default vectorization mode is int, this outputs integer indices (one per token).We can use other modes, for example binary for building bag of word models.</li></ul></p>

<p>We will build two modes binary and int to get the touch of these. At first we use binary to build a bag-of-words model.</p>

In [15]:
#Setting vocabulary size to 10000
VOCABULARY_SIZE = 10000

#Constructing binary vectorization layer
binary_vectorize_layer = TextVectorization(
    max_tokens=VOCABULARY_SIZE,
    output_mode='binary')

<p>In int mode, addition to maximum vocabulary size, we need to set an explicit maximum sequence length, which will cause the layer to pad or truncate sequences to exactly sequence_length_values.</p>

In [16]:
MAXIMUM_SEQUENCE_LENGTH = 250

#Constructing int vectorization layer with maximum sequence length of 250
int_vectorize_layer = TextVectorization(
    max_tokens=VOCABULARY_SIZE,
    output_mode='int',
    output_sequence_length=MAXIMUM_SEQUENCE_LENGTH)

<p>We will call <b>adapt</b> to fix the state of the preprocessing layer to the dataset. This step will cause the model to build an index of strings to integers. <br>
Call adapt when we are using training data.(for test data would leak information).

In [17]:
# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda text, labels: text)

#Calling adapt for binary int vectorized layers
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

<p>See the result of using these layers to preprocess data</p>


In [18]:
#Defining functions for binary and int vectorizd text
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [19]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [20]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of ex

In [21]:
print("'binary' vectorized question:", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [22]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[ 107   16    4 1139   37  184   52   47   16    1  215  417  518   12
    32  182    4   13   26    3 1416  227   11  894  627   52  309  718
    16 1274  100    5  244 1826    8  119  224   95   97  657  858    1
   797   20  125  117  559 2282  194   73    9   23   30   50    3  115
   566   10   97   59   73  204    4  412   16    4 1139   45 1670  601
     5  694   38    4   14  851   37  967   97   98   65 2374    9  132
    45   55   11  227  401    2  244 1826   72  534  156  967    8 7808
    32   65  468   11  181 2332  198 1461    4    1 1093  447  309  100
    71   65  454  109    5 3560  224   20  139   36   11 9052   37  233
    14    2    1  224  107   12  137   76  779    3  102   14    3  176
  1853    2  332    4    5   39    4  787   12 1320 3579   26  106    1
     2   98   65    8 2902    2  244    1   69   33  336    1    5 1759
     9 1157 2531    9  568  729   12    1    4  321   13    2 1826   52
    47    2    1  244 1826

<p>In the above binary and int vectorized text results, binary mode return the array denoting which tokens exits at least once in the input. int mode replaces each token by an integer, thus preservating their order. <br>
By calling <b>.get_vocabulary()</b> We can lookup the token (string) that each integer corresponds.

In [23]:
#Printing the token of int vectorized layer at 1152th and 567th line and the vecabulary size
print("1152 ---> ", int_vectorize_layer.get_vocabulary()[1152])
print("567 ---> ", int_vectorize_layer.get_vocabulary()[567])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1152 --->  seen
567 --->  isnt
Vocabulary size: 10000


<p>The last preprocessing step, we will apply the <b>TextVectorization</b> layers we created previously to the train, validation and test dataset.</p>

In [24]:
#Applying binary vectorization to three formats
binary_train_ds = train_ds.map(binary_vectorize_text)
binary_val_ds = val_ds.map(binary_vectorize_text)
binary_test_ds = test_ds.map(binary_vectorize_text)

#Applying int vectorization to three formats
int_train_ds = train_ds.map(int_vectorize_text)
int_val_ds = val_ds.map(int_vectorize_text)
int_test_ds = test_ds.map(int_vectorize_text)

<h4>Configure the dataset for performance.</h4>
<p>The two important methods we should use when loading data to make sure that I/O does not go to blocking state are cache() and prefetch().
<ul><li><b>.cache()</b> it will keeps data in memory after it's loaded off disk. This method will ensure that dataset does not become a bottlneck while training the model. If the dataset is very large to fit into memory, we can also use cache() to create a performant on-disk cache, which is more efficient than many small files.</li>
<li><b>.prefetch()</b> it will overlaps data prepocessing and model execution while training.</li>
</p>


In [25]:
AUTOTUNE = tf.data.AUTOTUNE

#defining the function for configuring the dataset
def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [26]:
#Applying the configure_dataset() to all formats
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

<h3>Train the model</h3>
<p>We will create a neural network. For the binary vectorized data, we train a simple bag-of-words linear model:</p>

In [27]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 5s 20ms/step - loss: 1.2451 - accuracy: 0.5145 - val_loss: 0.9180 - val_accuracy: 0.7756
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.8235 - accuracy: 0.8141 - val_loss: 0.7528 - val_accuracy: 0.7975
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.6549 - accuracy: 0.8574 - val_loss: 0.6664 - val_accuracy: 0.8131
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.5546 - accuracy: 0.8837 - val_loss: 0.6124 - val_accuracy: 0.8225
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.4851 - accuracy: 0.9007 - val_loss: 0.5754 - val_accuracy: 0.8306
Epoch 6/10
200/200 [==============================] - 1s 4ms/step - loss: 0.4326 - accuracy: 0.9133 - val_loss: 0.5485 - val_accuracy: 0.8338
Epoch 7/10
200/200 [==============================] - 1s 4ms/step - loss: 0.3910 - accuracy: 0.9246 - val_loss: 0.5283 - val_accuracy: 0.8388
Epoch

<p>We will use int vectorized layer as we constructed previously to build a one-dimension Convolutional network.</p>

In [28]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [29]:
# vocabulary_size is VOCABULARY_SIZE + 1 since 0 is used additionally for padding.
int_model = create_model(vocab_size=VOCABULARY_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 8s 38ms/step - loss: 1.3131 - accuracy: 0.3922 - val_loss: 0.7756 - val_accuracy: 0.6963
Epoch 2/5
200/200 [==============================] - 5s 25ms/step - loss: 0.7028 - accuracy: 0.7195 - val_loss: 0.5334 - val_accuracy: 0.8062
Epoch 3/5
200/200 [==============================] - 5s 25ms/step - loss: 0.4268 - accuracy: 0.8592 - val_loss: 0.4643 - val_accuracy: 0.8269
Epoch 4/5
200/200 [==============================] - 5s 25ms/step - loss: 0.2478 - accuracy: 0.9362 - val_loss: 0.4562 - val_accuracy: 0.8269
Epoch 5/5
200/200 [==============================] - 5s 26ms/step - loss: 0.1296 - accuracy: 0.9756 - val_loss: 0.4799 - val_accuracy: 0.8263


<p>We compare two models</p>


In [30]:
#Displaying the summary of binary model
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
#Displaying the summary of int model
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
#Evaluating two models and printing the accuracy
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 5s 18ms/step - loss: 0.5149 - accuracy: 0.8135
Binary model accuracy: 81.56%
Int model accuracy: 81.35%
